In [6]:
# !pip install pydata-google-auth 
# !pip install --upgrade google-cloud-bigquery[pandas]
# !pip install pyarrow
# !pip install pandas-gbq

In [1]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/bigquery'])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=JJcSBjLrv42FqItNKT6V0CbRSXcHCt&access_type=offline


Unable to create credentials directory.


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Instantiate the BigQuery client (using project ID and credentials)
from google.cloud import bigquery
project_id = "etsy-bigquery-adhoc-prod"
client = bigquery.Client(project=project_id, credentials=credentials)

### latest return profile on active traders

In [35]:
query1 = """
with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- active incorporated sellers with active listings 
active_traders as (select distinct l.shop_id,
l.user_id,
sb.seller_tier_new,
sb.country_name -- where the shop is based at
-- l.is_active,
-- sb.user_state,
-- sb.shop_status,
-- s.trader_distinction
from `etsy-data-warehouse-prod`.listing_mart.listings l 
inner join `etsy-data-warehouse-prod`.rollups.seller_basics sb
on l.shop_id = sb.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s
on sb.shop_id = s.shop_id
where l.is_active = 1 -- active listings only
and sb.user_state = 'active' -- active users only
and sb.shop_status = 'active' -- active shops only
and s.trader_distinction = 1  -- incorporated sellers only
-- and l.shop_id in (5928292, 5447890, 15903859, 20456727, 23763260, 29784414)
),
-- select count(distinct user_id), count(*) from active_traders --110912, 110912


-- get shipping profile shops (all active traders with active listings should be included in this table)
shipping_profile as (select distinct a.shop_id,
a.user_id,
-- a.country_name as seller_base_country, 
-- sp.origin_country_id, -- shipping origin country id
-- countries.name as shipping_country,
-- sp.ships_everywhere,
min(case when a.country_name in (select name from eu_countries) then '1-EU Origin' 
     when a.country_name not in (select name from eu_countries) and ships_everywhere = 1 then '2-Non EU Origin/Ships to EU'
     else 'Non EU related' end) as ship_profile
from active_traders a
left join `etsy-data-warehouse-prod.etsy_shard.shipping_profile`  sp
on a.shop_id = sp.shop_id
left join `etsy-data-warehouse-prod.etsy_v2.countries` countries
on sp.origin_country_id = countries.country_id
where sp.is_deleted = 0 -- active shipping profile only
-- and a.shop_id in (23077606, 34769184, 22176230, 31070274, 29949827)
group by a.shop_id, a.user_id)

-- select count(distinct shop_id), count(*) from shipping_profile -- 110912, 110912


-- get latest return policy on active traders
select shop_id,
seller_tier_new,
return_items_within_days
from (select a.shop_id,
        a.seller_tier_new,
        b.return_items_within_days,
        timestamp_seconds(b.create_date) as create_date,
        row_number() over (partition by a.shop_id order by timestamp_seconds(b.create_date) desc) as rnk --there are shops that have two records that have the same create date and seconds
      from active_traders a 
      left join `etsy-data-warehouse-prod`.etsy_shard.shop_structured_policies b
      on a.shop_id = b.shop_id
      --  where shop_id in (5447890, 15903859, 20456727, 23763260, 29784414)
     ) 
where rnk = 1
"""

In [36]:
query_job = client.query(query1)
latest_return_profile = pd.read_gbq(query=query1, project_id=project_id, 
                                    credentials=credentials, progress_bar_type='tqdm')

Downloading: 100%|█████████████████| 110913/110913 [00:02<00:00, 53166.82rows/s]


In [37]:
latest_return_profile.shape

(110913, 3)

In [38]:
latest_return_profile.head(10)

,shop_id,seller_tier_new,return_items_within_days
0,22163179,Empty Shop,<NA>
1,18639381,Non Active Shop,30
2,27851318,Non Active Shop,30
3,27318231,Empty Shop,<NA>
4,33831470,Empty Shop,<NA>
5,25801140,Non Active Shop,30
6,32639826,Closed Shop,<NA>
7,33842455,Empty Shop,<NA>
8,33824912,Empty Shop,<NA>
9,35516404,Empty Shop,<NA>


In [40]:
latest_return_profile['shop_id'].nunique()

110913

In [45]:
# number of shops without structured return policy
check1 = latest_return_profile[latest_return_profile['return_items_within_days'].isnull()]
check1.shape

(58572, 3)

### Structured return policy on active traders

In [26]:
query2 = """
with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- active incorporated sellers with active listings 
active_traders as (select distinct l.shop_id,
l.user_id,
sb.seller_tier_new,
sb.country_name -- where the shop is based at
-- l.is_active,
-- sb.user_state,
-- sb.shop_status,
-- s.trader_distinction
from `etsy-data-warehouse-prod`.listing_mart.listings l 
inner join `etsy-data-warehouse-prod`.rollups.seller_basics sb
on l.shop_id = sb.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s
on sb.shop_id = s.shop_id
where l.is_active = 1 -- active listings only
and sb.user_state = 'active' -- active users only
and sb.shop_status = 'active' -- active shops only
and s.trader_distinction = 1  -- incorporated sellers only
-- and l.shop_id in (5928292, 5447890, 15903859, 20456727, 23763260, 29784414)
),
-- select count(distinct user_id), count(*) from active_traders --110912, 110912


-- get shipping profile shops (all active traders with active listings should be included in this table)
shipping_profile as (select distinct a.shop_id,
a.user_id,
-- a.country_name as seller_base_country, 
-- sp.origin_country_id, -- shipping origin country id
-- countries.name as shipping_country,
-- sp.ships_everywhere,
min(case when a.country_name in (select name from eu_countries) then '1-EU Origin' 
     when a.country_name not in (select name from eu_countries) and ships_everywhere = 1 then '2-Non EU Origin/Ships to EU'
     else 'Non EU related' end) as ship_profile
from active_traders a
left join `etsy-data-warehouse-prod.etsy_shard.shipping_profile`  sp
on a.shop_id = sp.shop_id
left join `etsy-data-warehouse-prod.etsy_v2.countries` countries
on sp.origin_country_id = countries.country_id
where sp.is_deleted = 0 -- active shipping profile only
-- and a.shop_id in (23077606, 34769184, 22176230, 31070274, 29949827)
group by a.shop_id, a.user_id)

-- select count(distinct shop_id), count(*) from shipping_profile -- 110912, 110912


-- get structured return policy on these active traders
select a.shop_id,
a.seller_tier_new,
p.return_items_within_days
from active_traders a
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s 
on a.shop_id = s.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_structured_policies p
on s.structured_policies_id = p.structured_policies_id
"""

In [27]:
query_job = client.query(query2)
structured_return_profile = pd.read_gbq(query=query2, project_id=project_id, 
                                    credentials=credentials, progress_bar_type='tqdm')

Downloading: 100%|█████████████████| 110913/110913 [00:02<00:00, 47489.90rows/s]


In [34]:
structured_return_profile.shape

(110913, 3)

In [30]:
structured_return_profile.head(10)

,shop_id,seller_tier_new,return_items_within_days
0,27851318,Non Active Shop,30
1,27806463,Empty Shop,<NA>
2,25845514,Non Active Shop,<NA>
3,25827067,Non Active Shop,30
4,35060897,Empty Shop,<NA>
5,32639826,Closed Shop,<NA>
6,33824912,Empty Shop,<NA>
7,33831470,Empty Shop,<NA>
8,33842455,Empty Shop,<NA>
9,35696151,Empty Shop,<NA>


In [55]:
latest_return_profile['return_items_within_days'].describe()

count    52341.000000
mean        25.372863
std          9.180874
min          7.000000
25%         21.000000
50%         30.000000
75%         30.000000
max         90.000000
Name: return_items_within_days, dtype: float64

In [54]:
structured_return_profile['return_items_within_days'].describe()

count    52335.000000
mean        25.373249
std          9.180828
min          7.000000
25%         21.000000
50%         30.000000
75%         30.000000
max         90.000000
Name: return_items_within_days, dtype: float64

In [44]:
check2 = structured_return_profile[structured_return_profile['return_items_within_days'].isnull()]
check2.shape

(58578, 3)

In [52]:
check2[~check2['shop_id'].isin(check1['shop_id'])]

,shop_id,seller_tier_new,return_items_within_days
22913,9966073,Power Shop,<NA>
25040,11131258,Power Shop,<NA>
31745,31570633,Small Shop,<NA>
33181,35369612,Small Shop,<NA>
64620,34778172,Listed Shop,<NA>
107108,20142864,Medium Shop,<NA>


In [50]:
latest_return_profile[latest_return_profile['shop_id'].isin([9966073, 11131258, 31570633, 35369612, 34778172, 20142864])]

,shop_id,seller_tier_new,return_items_within_days
22611,9966073,Power Shop,30
24391,11131258,Power Shop,7
36313,31570633,Small Shop,21
55919,35369612,Small Shop,30
62828,34778172,Listed Shop,14
105462,20142864,Medium Shop,30


In [9]:
query = """with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- to get if a shop has structured return policy or not and how many days
return_policy as (
select distinct
l.shop_id, 
p.return_items_within_days  -- to get how many days allowed for return to define structured policy, structured policy means store has defined return policy
from
  `etsy-data-warehouse-prod`.listing_mart.listings l
  inner join `etsy-data-warehouse-prod`.rollups.seller_basics b
    on l.shop_id = b.shop_id
  left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s  -- middle table to connect shop_id to structured_policies_id
    on l.shop_id = s.shop_id
  left join `etsy-data-warehouse-prod`.etsy_shard.shop_structured_policies p
    on s.structured_policies_id = p.structured_policies_id
where
    l.is_active = 1  -- why filter on listing is active? We only want to target shops with active listings that are traders who went through KYB
    and b.active_seller_status = 1  
    and p.return_items_within_days is not null  --why?
    and return_policy_type = "structured"  --why?
)

select seller_tier_new, 
seller_country,ship_profile, 
return_items_within_days,
count(distinct a.user_id) as total_sellers 
from (
      select a.seller_tier_new, 
      a.user_id, 
      f.return_items_within_days,
      case when a.country_name in (
            'Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal', 'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece') then 'EU Seller' else 'Other Seller' end as seller_country,
      min(case when d.country_id is not null and c.ships_everywhere = 0 then '2-EU Origin Ship Country/Ships Locally' 
      when c.ships_everywhere =1 then '1-EU Origin/Ships Everywhere' else '3-Doesnt Ship Everywhere' end) as ship_profile – identify sellers who ship to EU
      from `etsy-data-warehouse-prod.rollups.seller_basics` A
      inner join `etsy-data-warehouse-prod.etsy_payments.shop_business_verification_attempts` b  -- why this table, why not KYC_Checks table?         This is the KYB table                                                   
      on a.shop_id = b.shop_id                                                  -- inner join to filter out shops that didn't go through KYB?
      inner join `etsy-data-warehouse-prod.etsy_shard.shipping_profile` c       -- inner join to filter out shops with no shipping profile? –Laws are applicable to traders who ships, filtering out people who sell digital product, we don’t need to display message to them.
      on a.shop_id = c.shop_id and is_deleted = 0  -- keep only active shipping profile
      left join eu_countries d
      on c.origin_country_id = d.country_id
      left join return_policy f
      on a.shop_id = f.shop_id
      where a.user_state = 'active' and a.shop_status = 'active'  -- why both have to be active?
      group by 1,2,3,4
      )A
group by 1,2,3,4
"""

In [ ]:
new_query = """
with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- active incorporated sellers with active listings 
active_traders as (select distinct l.shop_id,
l.user_id,
sb.country_name -- where the shop is based at
-- l.is_active,
-- sb.user_state,
-- sb.shop_status,
-- s.trader_distinction
from `etsy-data-warehouse-prod`.listing_mart.listings l 
inner join `etsy-data-warehouse-prod`.rollups.seller_basics sb
on l.shop_id = sb.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s
on sb.shop_id = s.shop_id
where l.is_active = 1 -- active listings only
and sb.user_state = 'active' -- active users only
and sb.shop_status = 'active' -- active shops only
and s.trader_distinction = 1  -- incorporated sellers only
-- and l.shop_id in (5928292, 5447890, 15903859, 20456727, 23763260, 29784414)
),
-- select count(distinct user_id), count(*) from active_traders --110894, 110894

-- get latest return policy on active traders
latest_return_policies as (select shop_id,
return_items_within_days
from (select a.shop_id,
        b.return_items_within_days,
        timestamp_seconds(b.create_date) as create_date,
        row_number() over (partition by a.shop_id order by timestamp_seconds(b.create_date) desc) as rnk --there are shops that have two records that have the same create date and seconds
      from active_traders a 
      inner join `etsy-data-warehouse-prod`.etsy_shard.shop_structured_policies b
      on a.shop_id = b.shop_id
      --  where shop_id in (5447890, 15903859, 20456727, 23763260, 29784414)
     ) 
where rnk = 1
)


-- get shipping profile on shops
select distinct a.shop_id,
a.user_id,
a.country_name as seller_base_country, 
-- sp.origin_country_id, -- shipping origin country id
countries.name as shipping_country,
sp.is_deleted
from active_traders a
left join `etsy-data-warehouse-prod.etsy_shard.shipping_profile`  sp
on a.shop_id = sp.shop_id
left join `etsy-data-warehouse-prod.etsy_v2.countries` countries
on sp.origin_country_id = countries.country_id
where sp.is_deleted = 0 -- active shipping profile only
and a.shop_id in (23077606, 34769184, 22176230, 31070274, 29949827)
order by shop_id



"""

In [10]:
query_job = client.query(query)

In [14]:
er_main = pd.read_gbq(query=query, project_id=project_id, credentials=credentials,
           progress_bar_type='tqdm')

Downloading: 100%|███████████████████████████| 20/20 [00:00<00:00, 122.06rows/s]


In [15]:
er_main

,user_id,is_guest,mapped_user_id,country_id,iso_country_code,country_name,source,first_source_id,last_source_id,first_source_date,last_source_date,source_frequency
0,0,0,<NA>,79,CA,Canada,shop_setting_id,50008426647,50008426647,1471999508,1471999508,1
1,0,0,<NA>,209,US,United States,shop_setting_id,52862078865,52862078865,1475519975,1475519975,1
2,93,0,93,209,US,United States,shipping_profile_id,84029125,84031628,1347004616,1347004617,17
3,93,0,93,209,US,United States,login_session_id,857543350195,1052628326780,1590336015,1645802952,5
4,93,0,93,74,BR,Brazil,shipping_profile_id,84032072,84032072,1347004617,1347004617,1
5,93,0,93,209,US,United States,card_id,10043483853,959731294232,1392480330,1601478913,3
6,93,0,93,55,AF,Afghanistan,shipping_profile_id,84032954,84032954,1347004617,1347004617,3
7,93,0,93,209,US,United States,address_id,303647641,1048596269769,1311906752,1630113139,7
8,94,0,94,209,US,United States,card_id,1904783698,1904783698,1342559603,1342559603,1
9,94,0,94,209,US,United States,bank_account_id,25715127,25715127,1342559418,1342559418,1
